In [0]:
COAE：中文数据集

中文数据处理：
1、数据（去停用词）、分词、标词性、句法分析，
2、对未登录词进行均匀分布随机化
3、train与test同时统计

#词向量、词性向量、位置向量、句法结构


In [0]:
#中文数据集 COAE
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torch.optim as optim
import torch.autograd as autograd
#import torchtext.vocab as torchvocab
from torch.autograd import Variable
import tqdm
import os
import time
import re
import pandas as pd
import string
import gensim
#from gensim.models import word2vec
import time
import random
import nltk
import snowballstemmer
import collections
from collections import Counter
from nltk.corpus import stopwords
from itertools import chain
from sklearn.metrics import accuracy_score
import pyltp
from pyltp import Segmentor
from pyltp import Postagger
from pyltp import Parser
num_epochs = 200  #训练迭代次数
embed_size = 300  #词向量维度
num_hiddens = 128 #隐藏层神经单元个数
num_layers = 2    #隐藏层层数
bidirectional = True  #False为LSTM,True为双向LSTM
#batch_size = 64    #批处理大小
labels = 2          #二分类

In [4]:
!pip install pyltp

     |████████████████████████████████| 5.3MB 45.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/fc/3a/35/b11293efb2c77c0e7b6fa574271d51cddd9abd1f634535343c
Successfully built pyltp


In [0]:
 !pip install torch==0.4

    100% |████████████████████████████████| 484.0MB 29kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5c508000 @  0x7f5a2271e2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [0]:
!pip install snowballstemmer

In [0]:
!pip install gensim

In [0]:
#Hownet  （词性）
def Tag_wordslist(filepath):
    Tag_words = [line.strip() for line in open(filepath,'r',encoding = 'utf-8').readlines()]
    return Tag_words
CAdverbswords = Tag_wordslist('drive/python3/COAE/Chinese_tag/Adverbs(C).txt')#加载程度副词的路径
CNegativeRewords = Tag_wordslist('drive/python3/COAE/Chinese_tag/NegativeRe(C).txt')#加载负面评价词的路径
CNegativeSenwords = Tag_wordslist('drive/python3/COAE/Chinese_tag/NegativeSen(C).txt')#加载负面情感词的路径
CPositiveRewords = Tag_wordslist('drive/python3/COAE/Chinese_tag/PositiveRe(C).txt')#加载正面评价词的路径
CPositiveSenwords = Tag_wordslist('drive/python3/COAE/Chinese_tag/PositiveSen(C).txt')#加载正面情感词的路径
CInverwords = Tag_wordslist('drive/python3/COAE/Chinese_tag/Inver(C).txt')#加载否定情感词的路径
#stopwords = Tag_wordslist('drive/python3/COAE/Chinese_tag/stopwords_sum.txt')#加载停用词的路径


#COAE   读取数据路径，标注分类类别
def readCOAE_Pos():
    data = []
    files= open("drive/python3/COAE/COAE6千不分/pos_1.txt", "r",encoding = 'utf-8')  #drive/python3/COAE/train/pos.txt
    lines = files.readlines()
    for line in lines:
        data.append([line,1])
    return data
def readCOAE_Neg():
    data = []
    files= open("drive/python3/COAE/COAE6千不分/neg_-1.txt", "r",encoding = 'utf-8')
    lines = files.readlines()
    for line in lines:
        data.append([line,0])   
    return data

Pos = readCOAE_Pos()
Neg = readCOAE_Neg()
random.shuffle(Pos) #随机打乱数据
random.shuffle(Neg) #随机打乱数据
train_data = []
test_data = []
train_data = Pos[664:]+Neg[655:]
test_data = Pos[:664]+Neg[:655]
random.shuffle(train_data)
random.shuffle(test_data)



In [7]:

#合工大LTP分词、词性、句法分析
def LTP(data):
    LTP_DATA_DIR = 'drive/python3/COAE/ltp_data_v3.4.0'       #ltp模型目录的路径
    cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')  # 分词模型路径，模型名称为`cws.model`
    segmentor = Segmentor()  # 初始化实例
    segmentor.load(cws_model_path)  # 加载模型
    #加载词性相关文件
    pos_model_path = os.path.join(LTP_DATA_DIR, 'pos.model')  # 词性标注模型路径，模型名称为`pos.model`
    postagger = Postagger() # 初始化实例
    postagger.load(pos_model_path)  # 加载模型
    #加载句法分析相关文件
    par_model_path = os.path.join(LTP_DATA_DIR, 'parser.model')  # 依存句法分析模型路径，模型名称为`parser.model`
    parser = Parser() # 初始化实例
    parser.load(par_model_path)  # 加载模型
    datas = []#记录单词
    pos_tag = []#记录所有词性
    parserss = []#记录句法分析
    for review, score in data:
        #pyltp分词
        words = segmentor.segment(review)  # 分词
        words= " ".join(words).split()
        data = []
        for word in words:
            #if word not in stopwords:#去停用词
            data.append(word)    
        #pylt词性标注 
        postags = postagger.postag(data)  # 词性标注
        postags= ' '.join(postags).split()
        pos_tag.append(postags)
                #依存句法分析
        arcs = parser.parse(data, postags)  # 句法分析
        parsers = []
        for arc in arcs:
            parsers.append(arc.relation)   
            
        datas.append(data) 
        parserss.append(parsers)
    segmentor.release()  # 释放模型 
    postagger.release() 
    parser.release()  
    return datas,pos_tag,parserss

train_datas = []#记录单词
train_tag = []#记录所有词性
train_parserss = []#记录句法分析

train_datas,train_tag,train_parserss = LTP(train_data)

#词
tr_te_word = train_datas
vocab_word = set(chain(*tr_te_word)) #对所有词进行去重
vocab_word_size = len(vocab_word) #统计所以词的个数(不计重复)
word_to_idx = {word: i+1 for i, word in enumerate(vocab_word)}  #对去重后的所有词进行重新整理，标注序号，word：id
word_to_idx['<unk>'] = 0  
idx_to_word = {i+1: word for i, word in enumerate(vocab_word)}  #id：word
idx_to_word[0] = '<unk>'

#句法依存
tr_te_par = train_parserss
vocab_par = set(chain(*tr_te_par))
vocab_par_size = len(vocab_par)
par_to_idx = {word: i+1 for i, word in enumerate(vocab_par)}
par_to_idx['<unk>'] = 0
idx_to_par= {i+1: word for i, word in enumerate(vocab_par)}
idx_to_par[0] = '<unk>'

#加载词向量，300维
wvmodel = gensim.models.KeyedVectors.load_word2vec_format('drive/python3/COAE/sgns.target.word-character.char1-2.dynwin5.thr10.neg5.dim300.iter5',binary=False)  #300b中文 
print("end")
wordsList = np.load('drive/python3/COAE/wordsList_Chinese.npy') #加载已经整理好的词List 索引
wordsList = wordsList.tolist() 
embed_size_word = 300  #词向量维度
embed_size_tag = 30   #词性向量维度
weight_word = torch.zeros(vocab_word_size+3, embed_size_word)   #根据COAE数据集中每个词的id，重新生成的词向量数组 
weight_tag = torch.zeros(vocab_word_size+3, embed_size_tag)     #词性向量数组，同上

embed_word = nn.Embedding(vocab_word_size+2,embed_size_word) #对于未登录词，进行均匀分布（-0.1，0.1）之间随机初始化
embed_word.weight.data.uniform_(-0.1, 0.1)

embedd_tag = nn.Embedding(vocab_word_size+3,embed_size_tag) #随机生成一个词性向量，生成范围在（-0.1,0.1）之间
embedd_tag.weight.data.uniform_(-1.0, 1.0)

for i in range(len(word_to_idx)):
    indstr = idx_to_word[i]
    indid = word_to_idx[idx_to_word[i]]
    if indstr in wordsList:
        weight_word[indid, :] = torch.from_numpy(wvmodel.get_vector(indstr))
    else:
        weight_word[indid, :] = embed_word.weight[indid]#未登录词        
    if indstr in CAdverbswords or indstr in CNegativeRewords or indstr in CNegativeSenwords or indstr in CPositiveRewords or indstr in CPositiveSenwords or indstr in CInverwords:
        weight_tag[indid, :] = 1.2*embedd_tag.weight[indid]#根据HowNet，来重点标注词（重点词乘以1.2）
    else:
        weight_tag[indid, :] = embedd_tag.weight[indid]
weight_word[vocab_word_size+1, :] = embed_word.weight[vocab_word_size+1]#未登录词  
weight_tag[vocab_word_size+1, :] = 1.2*embedd_tag.weight[vocab_word_size+1]
weight_tag[vocab_word_size+2, :] = embedd_tag.weight[vocab_word_size+2]

end


In [0]:
pos_size = vocab_word_size+3
pos_dim = 25
tag_size = vocab_word_size+3
tag_dim = 30
parser_size = vocab_word_size+3
parser_dim = 25
batch_size = 16 
#将COAE数据集中每一条样例，进行词语与ID 转化
def encode_samples(train_datas,train_tag,train_parserss,maxlen=161):#词特征、位置特征、词性、句法
    features = []#词向量
    Lists = []#位置
    Li_tags = []#词性
    Li_pars = []#句法
    for sample in train_datas:#词向量、位置、词性
        feature = [] 
        List = []
        Li_tag = []#词性
        c = 161
        for token in sample:#token为一条评论文本里的一个单词
            if token in word_to_idx:
                feature.append(word_to_idx[token])#词
                Li_tag.append(word_to_idx[token])#词性
                p = sample.index(token)-len(sample)+maxlen+1  #位置特征向量
                List.append(p)
            else:
                feature.append(vocab_word_size+1) 
                if token in CAdverbswords or token in CNegativeRewords or token in CNegativeSenwords or token in CPositiveRewords or token in CPositiveSenwords or token in CInverwords:
                    Li_tag.append(vocab_word_size+1)
                else:
                    Li_tag.append(vocab_word_size+2)
             
                List.append(0)
                
        features.append(feature)
        Lists.append(List)
        Li_tags.append(Li_tag)

    for sample in train_parserss:#句法
       Li_par = []
       for token in sample:#token为一条评论文本里的一个单词
            if token in par_to_idx:
                Li_par.append(par_to_idx[token])
            else:
                Li_par.append(0)
       Li_pars.append(Li_par) 
    return features,Lists,Li_tags,Li_pars   

def pad_samples(features, maxlen=50, PAD=0):#词，每一条样例中，词语个数只保留前50个词，长度大于50直接去掉，小于50，以0填充
    padded_features = []
    for feature in features:
        if len(feature) >= maxlen:
            padded_feature = feature[:maxlen]
            
        else:
            padded_feature = feature
            while(len(padded_feature) < maxlen):
                padded_feature.append(PAD)
        padded_features.append(padded_feature)
    return padded_features
  
def pad_list(Lists, maxlen=50, PAD=0):#位置，与词一样
    padded_Lists = []  #位置ID
    for List in Lists:
        if len(List) >= maxlen:
            padded_List = List[:maxlen]
        else:
            padded_List = List
            while(len(padded_List) < maxlen):
                padded_List.append(PAD)
        padded_Lists.append(padded_List)
    return padded_Lists

def pad_tag(tags, maxlen=50, PAD=0):#词性，一样
    padded_Tag = []  #词性ID
    for tag in tags:
        if len(tag) >= maxlen:
            padded_t = tag[:maxlen]
        else:
            padded_t = tag
            while(len(padded_t) < maxlen):
                padded_t.append(PAD)
        padded_Tag.append(padded_t)
    return padded_Tag
 
def pad_par(parsers, maxlen=50, PAD=0):#句法，一样
    padded_Par = []  #词性ID
    for tag in parsers:
        if len(tag) >= maxlen:
            padded_p = tag[:maxlen]
        else:
            padded_p = tag
            while(len(padded_p) < maxlen):
                padded_p.append(PAD)
        padded_Par.append(padded_p)
    return padded_Par

#词向量、位置、词性、句法
train_fea,train_lis,train_tag,train_pars = encode_samples(train_datas,train_tag,train_parserss)#再传入词性
train_features = torch.LongTensor(pad_samples(train_fea))#词特征ID
train_Lists = torch.LongTensor(pad_list(train_lis))#位置特征ID
train_Tag = torch.LongTensor(pad_tag(train_tag))  #词性特征ID
train_parser = torch.LongTensor(pad_par(train_pars))  #句法
train_labels= torch.LongTensor([score for _, score in train_data])#标签


In [0]:
#层归一化
class LayerNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-6):
        super().__init__()
        self.eps = eps
       # self.hidden_size = hidden_size*3
        self.hidden_size = hidden_size
        self.weight = nn.Parameter(torch.ones(self.hidden_size))
        self.bias = nn.Parameter(torch.zeros(self.hidden_size))

    def forward(self, input):
        mu = torch.mean(input, dim=-1, keepdim=True)
        sigma = torch.std(input, dim=-1, keepdim=True).clamp(min=self.eps)
        output = (input - mu) / sigma
        return output * self.weight.expand_as(output) + self.bias.expand_as(output)

In [0]:
#Bi-LSTM 
import torch
import torch.nn as nn
import torch.nn.functional as F


class Model(nn.Module):
    def __init__(self,labels,embedding_dim,embedding_dim_tag,weight,tag,hidden_dim,pos_size,pos_dim,batch,num_layers,tag_size,tag_dim,parser_size,parser_dim):  
        super(Model,self).__init__()
        self.batch_size = batch #batch_size批次大小
        self.embedding_dim_1 = embedding_dim  #词向量维度 300b
     
        self.hidden_dim = hidden_dim  #隐藏神经单元个数 (128)
      
        self.labels = labels          #标签大小 2
        self.num_layers = num_layers  #隐藏层个数
        self.pos_size = pos_size      #位置特征行 pos_size = vocab_size
        self.pos_dim = pos_dim        #位置特征维度 25 
        self.tag_size = tag_size      #词性特征行 tag_size = vocab_size
        self.tag_dim = tag_dim        #词性特征维度 30 
        self.parser_size = parser_size  #句法特征行 parser_size = vocab_size
        self.parser_dim  = parser_dim   #句法特征维度 25
        self.bidirectional = True    #False为LSTM True为Bilstm
        self.num_directions = 2 if self.bidirectional else 1 
        self.embedding_1 = nn.Embedding.from_pretrained(weight) #引入词向量
        self.embedding_1.weight.requires_grad = False
        
        self.tag_embeds = nn.Embedding.from_pretrained(tag) #引入词性向量30(对情感词，否定词，副词*1.2权值)
        self.tag_embeds.weight.requires_grad = False
 
        self.pos_embeds = nn.Embedding(self.pos_size,self.pos_dim) #位置特征25（随机） 
        
        self.parser_embeds = nn.Embedding(self.parser_size,self.parser_dim)  #依存分析树25（随机）
      
        self.lstm_1 = nn.LSTM(input_size=self.embedding_dim_1+self.pos_dim,
                              hidden_size=self.hidden_dim,
                              num_layers=self.num_layers, 
                              bidirectional=self.bidirectional,
                              dropout=0,
                              batch_first = True
                             )#125b（词+位置）
        
        self.lstm_2 = nn.LSTM(input_size=self.embedding_dim_1+self.tag_dim,
                              hidden_size=self.hidden_dim,
                              num_layers=self.num_layers, 
                              bidirectional=self.bidirectional,
                              dropout=0,
                              batch_first = True
                             )#130b（词+词性）
        
        self.lstm_3 = nn.LSTM(input_size=self.embedding_dim_1+self.parser_dim,
                              hidden_size=self.hidden_dim,
                              num_layers=self.num_layers, 
                              bidirectional=self.bidirectional,
                              dropout=0,
                              batch_first = True
                             )#125b(词+依存分析)
        self.ln = LayerNorm(self.hidden_dim * self.num_directions) 
        self.Linear = nn.Linear(self.hidden_dim * self.num_directions*3,self.hidden_dim) #线性层768-》128
        self.logistic_1 = nn.Linear(self.hidden_dim,self.labels)#128-》2
        self.dropout=nn.Dropout(p=0.5)
        self.dropout_emb=nn.Dropout(p=0.3)
        self._init_weights()
    
    #初始化：pos_embeds、parser_embeds      
    def _init_weights(self, scope=1.):
        self.pos_embeds.weight.data.uniform_(-scope,scope)
        self.parser_embeds.weight.data.uniform_(-scope,scope)
 
    def forward(self,sentence,pos,tag,parse):
        embeds_1 = torch.cat(((self.embedding_1(sentence)),(self.pos_embeds(pos))),2) #词向量+位置 拼接 325
        embeds_2 = torch.cat(((self.embedding_1(sentence)) , (self.tag_embeds(tag))),2)  #词向量+词性 拼接 330
        embeds_3 = torch.cat(((self.embedding_1(sentence)) , (self.parser_embeds(parse))),2) #词向量+句法 拼接 325
 
        states_1, hidden = self.lstm_1(embeds_1)#Bi-LSTM
        states_2, hidden = self.lstm_2(embeds_2)#Bi-LSTM
        states_3, hidden = self.lstm_3(embeds_3)#Bi-LSTM
        
        #转置
        states_1 = torch.transpose(states_1,0,1)
        states_2 = torch.transpose(states_2,0,1)
        states_3 = torch.transpose(states_3,0,1)
        
        #tanh激励函数
        states_1 = F.tanh(states_1)
        states_2 = F.tanh(states_2)
        states_3 = F.tanh(states_3)
 
        #层归一化
        states_1 = self.ln(states_1)[-1]
        states_2 = self.ln(states_2)[-1]
        states_3 = self.ln(states_3)[-1]
       
        #tanh激励函数
        states_1 = F.tanh(states_1)
        states_2 = F.tanh(states_2)
        states_3 = F.tanh(states_3)
        
        output = torch.cat([states_1,states_2,states_3],1) #将三个通道结果，进行拼接，融合
        output = self.Linear(output)#线性层 768-》128
        output = F.relu(output)#relu激励
        return F.log_softmax(self.logistic_1(output))#128->2
        
       

In [24]:
#1
device = torch.device('cuda:0')
# 定义device，是否使用GPU，依据计算机配置自动会选择
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model= Model_1(labels,vocab_size+1,embed_size,embed_size_tag,weight,weight_tag,num_hiddens,pos_size,pos_dim,batch_size,num_layers)
model= Model(labels,embed_size,embed_size_tag,weight_word,weight_tag,num_hiddens,pos_size,pos_dim,batch_size,num_layers,tag_size,tag_dim,parser_size,parser_dim)
print(model)
model.to(device)
#model = model.cuda()
loss_function = nn.CrossEntropyLoss()# size_average默认情况下是True，对每个小批次的损失取平均值。 但是，如果字段size_average设置为False
#optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.3,weight_decay=3e-8)
#optimizer = optim.SGD(model.parameters(), lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

#optimizer = optim.Adadelta(filter(lambda p: p.requires_grad, model.parameters()), lr=0.01, rho=0.9, eps=1e-06, weight_decay=0.001)#filter(lambda p: p.requires_grad, model.parameters())

#optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
optimizer = optim.Adagrad(model.parameters(), lr=0.01,  weight_decay=0.001)
#optimizer = optim.Adadelta(model.parameters(), lr=0.01, rho=0.9, eps=1e-06, weight_decay=0.001)#Adam
#optimizer = optim.Adagrad([
#                       {'params': model.embedding_1.parameters(),'lr':0.009}
#                      ], lr=0.09,weight_decay=5e-4)


Model(
  (embedding_1): Embedding(17620, 300)
  (tag_embeds): Embedding(17620, 30)
  (pos_embeds): Embedding(17620, 25)
  (parser_embeds): Embedding(17620, 25)
  (lstm_1): LSTM(325, 128, num_layers=2, batch_first=True, bidirectional=True)
  (lstm_2): LSTM(330, 128, num_layers=2, batch_first=True, bidirectional=True)
  (lstm_3): LSTM(325, 128, num_layers=2, batch_first=True, bidirectional=True)
  (ln): LayerNorm()
  (Linear): Linear(in_features=768, out_features=128, bias=True)
  (logistic_1): Linear(in_features=128, out_features=2, bias=True)
  (dropout): Dropout(p=0.5)
  (dropout_emb): Dropout(p=0.3)
)


In [0]:
#1
train_set = torch.utils.data.TensorDataset(train_features,train_Lists,train_Tag,train_parser,train_labels)
#test_set = torch.utils.data.TensorDataset(test_features,test_Lists,test_Tag,test_parser,test_labels)

train_iter = torch.utils.data.DataLoader(train_set, batch_size=batch_size,shuffle=True)
#test_iter = torch.utils.data.DataLoader(test_set, batch_size=batch_size,shuffle=False)



In [0]:
test_datas = []#记录单词
test_tag = []#记录所有词性
test_parserss = []#记录句法分析
test_datas,test_tag,test_parserss = LTP(test_data)

test_fea,test_lis,test_tag,test_pars = encode_samples(test_datas,test_tag,test_parserss)
test_features = torch.LongTensor(pad_samples(test_fea))
test_Lists = torch.LongTensor(pad_list(test_lis))
test_Tag = torch.LongTensor(pad_tag(test_tag))  #词性特征ID
test_parser = torch.LongTensor(pad_par(test_pars))  #句法
test_labels= torch.LongTensor([score for _, score in test_data])

test_set = torch.utils.data.TensorDataset(test_features,test_Lists,test_Tag,test_parser,test_labels)
test_iter = torch.utils.data.DataLoader(test_set, batch_size=batch_size,shuffle=True)

In [27]:
num_epochs = 140#94
for epoch in range(num_epochs):
    print("epoch:%d" %epoch)
    acc=0 
    total=0
    running_loss=0.0
    test_losses=0.0
    n = 0.0
    m= 0.0
    model.train()
    for train_i in train_iter:
        n += 1
        x_train_wvm = Variable(train_i[0].cuda())
        x_train_pos = Variable(train_i[1].cuda())
        x_train_tag = Variable(train_i[2].cuda())
        x_train_parser = Variable(train_i[3].cuda())
        label = Variable(train_i[4].cuda())
        y = model(x_train_wvm,x_train_pos,x_train_tag,x_train_parser) #词向量+位置特征向量
        loss = loss_function(y, label)
        optimizer.zero_grad() #将参数的grad值初始化为0
        loss.backward()#
        optimizer.step()    
         
        y = np.argmax(y.cpu().data.numpy(),axis=1)
        running_loss += float(loss)
        for y1,y2 in zip(y,label):
            if y1==y2:
                acc+=1
            total+=1
    #print(n)
    #print(len(train_iter.dataset))
    print("loss:%.5f,train:%.4f %%" %(float(running_loss)/n,(100*float(acc)/total)))
 
   #acc_t = 0
   # total_t = 0
    count_predict = [0,0]
    count_total = [0,0]
    count_right = [0,0]
    test_acc = 0.0
    #with torch.no_grad():  # no grad when test and predict
    model.eval()
    #hidden = model.init_hidden()
    m = 0
    for test_i in test_iter:
        m += 1
       
        x_test_wvm = Variable(test_i[0].cuda())
        x_test_pos = Variable(test_i[1].cuda())
        x_test_tag = Variable(test_i[2].cuda())
        x_test_parser = Variable(test_i[3].cuda())
        test_label = Variable(test_i[4].cuda())
        
        yy = model(x_test_wvm,x_test_pos,x_test_tag,x_test_parser)
        
        test_loss = loss_function(yy, test_label)
        test_losses += test_loss
        y = np.argmax(yy.cpu().data.numpy(),axis=1)
        
        test_acc += accuracy_score(torch.argmax(yy.cpu().data,
                                                    dim=1), test_label.cpu())
        for y1,y2 in zip(y,test_label):
            count_predict[y1]+=1
            count_total[y2]+=1
            if y1==y2:
                count_right[y1]+=1

    #print(count_predict)
    #print(count_total)
   # print(count_right)
    precision = [0,0]
    recall = [0,0]
    for i in range(len(count_predict)):
        if count_predict[i]!=0:
            precision[i] = float(count_right[i])/count_predict[i]
            recall[i] = float(count_right[i])/count_total[i]
    
    precision = sum(precision)/len(precision)
    recall = sum(recall)/len(recall)    

    print("loss:%.5f , 准确率：%.3f , 召回率：%.3f , f：%.3f   | %.3f" %(test_losses.data/m,precision,recall,(2*precision*recall)/(precision+recall),test_acc/m))

epoch:0


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:101: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


loss:0.68023,train:56.5691 %
loss:0.64594 , 准确率：0.673 , 召回率：0.601 , f：0.635   | 0.603
epoch:1
loss:0.61525,train:62.8925 %
loss:0.46449 , 准确率：0.811 , 召回率：0.808 , f：0.810   | 0.809
epoch:2
loss:0.54121,train:73.3604 %
loss:0.41645 , 准确率：0.853 , 召回率：0.838 , f：0.846   | 0.836
epoch:3
loss:0.55582,train:69.8141 %
loss:0.48463 , 准确率：0.814 , 召回率：0.771 , f：0.792   | 0.768
epoch:4
loss:0.50481,train:73.8945 %
loss:0.51440 , 准确率：0.797 , 召回率：0.711 , f：0.752   | 0.711
epoch:5
loss:0.46195,train:77.3553 %
loss:0.52721 , 准确率：0.746 , 召回率：0.641 , f：0.689   | 0.643
epoch:6
loss:0.45363,train:78.1243 %
loss:0.42419 , 准确率：0.837 , 召回率：0.830 , f：0.833   | 0.830
epoch:7
loss:0.56235,train:68.8314 %
loss:0.66036 , 准确率：0.745 , 召回率：0.547 , f：0.631   | 0.543
epoch:8
loss:0.45985,train:78.5302 %
loss:0.41297 , 准确率：0.849 , 召回率：0.847 , f：0.848   | 0.847
epoch:9
loss:0.43989,train:81.0938 %
loss:0.50158 , 准确率：0.804 , 召回率：0.735 , f：0.768   | 0.733
epoch:10
loss:0.47866,train:76.1376 %
loss:0.44502 , 准确率：0.831 , 召回率

KeyboardInterrupt: ignored

In [0]:

Loss_list = []
Accuracy_list =[]#25
num_epochs = 200#94
for epoch in range(num_epochs):
    print("epoch:%d" %epoch)
    acc=0 
    total=0
    running_loss=0.0
    test_losses=0.0
    n = 0.0
    m= 0.0
    model.train()
   # Hidden = model.init_hidden()
    for train_i in train_iter:
        n += 1
       #cuda()
       # model.zero_grad()
        x_train_wvm = Variable(train_i[0].cuda())
        x_train_pos = Variable(train_i[1].cuda())
        x_train_tag = Variable(train_i[2].cuda())
        x_train_parser = Variable(train_i[3].cuda())
        label = Variable(train_i[4].cuda())
        y = model(x_train_wvm,x_train_pos,x_train_tag,x_train_parser) #词向量+位置特征向量
       
      #  l2_reg = 0
       # for param in model.parameters():
      #      l2_reg += torch.norm(param)
       # w,b =[param.item() for param in model.parameters()]
        
        loss = loss_function(y, label)
       # loss += reg_lambad * l2_reg
        optimizer.zero_grad() #将参数的grad值初始化为0
        loss.backward()#
        optimizer.step()    
         
        y = np.argmax(y.cpu().data.numpy(),axis=1)
        running_loss += float(loss)
        for y1,y2 in zip(y,label):
            if y1==y2:
                acc+=1
            total+=1
    #print(n)
    #print(len(train_iter.dataset))
    print("loss:%.5f,train:%.4f %%" %(float(running_loss)/n,(100*float(acc)/total)))
    Loss_list.append(float(running_loss)/total)
    Accuracy_list.append((100*float(acc)/total))
torch.save(model, "drive/python3/COAE/model_COAE_LSTM_1.pkl")
print ("model has been saved")
#model_COAE_BiLSTM 72.65

In [0]:
#test
count_predict = [0,0]
count_total = [0,0]
count_right = [0,0]
test_acc = 0.0
m = 0
net = torch.load("drive/python3/COAE/model_COAE_LSTM_1.pkl")

net.eval()
model.eval()
for test_i in test_iter:
    m += 1
    x_test_wvm = Variable(test_i[0].cuda())
    x_test_pos = Variable(test_i[1].cuda())
    x_test_tag = Variable(test_i[2].cuda())
    x_test_parser = Variable(test_i[3].cuda())
    test_label = Variable(test_i[4].cuda())
    yy = model(x_test_wvm,x_test_pos,x_test_tag,x_test_parser)
    test_loss = loss_function(yy, test_label)
    test_losses += test_loss
    y = np.argmax(yy.cpu().data.numpy(),axis=1)
    test_acc += accuracy_score(torch.argmax(yy.cpu().data, dim=1), test_label.cpu())
    for y1,y2 in zip(y,test_label):
        count_predict[y1]+=1
        count_total[y2]+=1
   #     print("%d,%d" %(y1,y2))
        if y1==y2:
            count_right[y1]+=1

precision = [0,0]
recall = [0,0]
for i in range(len(count_predict)):
    if count_predict[i]!=0:
        precision[i] = float(count_right[i])/count_predict[i]
        recall[i] = float(count_right[i])/count_total[i]
    
precision = sum(precision)/len(precision)
recall = sum(recall)/len(recall)    
print("loss:%.5f , 准确率：%.3f , 召回率：%.3f , f：%.3f   |准确率 %.3f" %(test_losses.data/m,precision,recall,(2*precision*recall)/(precision+recall),test_acc/m))


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:104: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


loss:0.53892 , 准确率：0.905 , 召回率：0.904 , f：0.905   |准确率 0.906


In [0]:
#中文8个样例处理
import codecs
def COAE_test_end():
    data = []
   # with codecs.open('drive/python3/COAE/pn.txt','r','utf-8') as f:#'drive/python3/COAE/COAE_col.txt
    with codecs.open('drive/python3/COAE/COAE_col.txt','r','utf-8') as f:
        for line in f:
            lines = line[2:]
            if line[0] =='1':
                data.append([lines,1])
            else:
                data.append([lines,0])
    return data
test_data_end = COAE_test_end()

test_datas_end = []#记录单词
test_tag_end = []#记录所有词性
test_parserss_end = []#记录句法分析
test_datas_end,test_tag_end,test_parserss_end = LTP(test_data_end)

test_fea_end,test_lis_end,test_tag_end,test_pars_end = encode_samples(test_datas_end,test_tag_end,test_parserss_end)

test_features_end = torch.LongTensor(pad_samples(test_fea_end))
test_Lists_end = torch.LongTensor(pad_list(test_lis_end))
test_Tag_end = torch.LongTensor(pad_tag(test_tag_end))  #词性特征ID
test_parser_end = torch.LongTensor(pad_par(test_pars_end))  #句法
test_labels_end= torch.LongTensor([score for _, score in test_data_end])

test_set_end = torch.utils.data.TensorDataset(test_features_end,test_Lists_end,test_Tag_end,test_parser_end,test_labels_end)
test_iter_end = torch.utils.data.DataLoader(test_set_end, batch_size=batch_size,shuffle=False)#False



In [0]:
#中文8个样例对比测试
count_predict = [0,0]
count_total = [0,0]
count_right = [0,0]
test_acc = 0.0
m = 0
net = torch.load("drive/python3/COAE/model_COAE_BiLSTM.pkl")
for test_i in test_iter_end:
    m += 1
    x_test_wvm = Variable(test_i[0].cuda())
    x_test_pos = Variable(test_i[1].cuda())
    x_test_tag = Variable(test_i[2].cuda())
    x_test_parser = Variable(test_i[3].cuda())
    test_label = Variable(test_i[4].cuda())
    yy = model(x_test_wvm,x_test_pos,x_test_tag,x_test_parser)
    test_loss = loss_function(yy, test_label)
    test_losses += test_loss
    y = np.argmax(yy.cpu().data.numpy(),axis=1)
    test_acc += accuracy_score(torch.argmax(yy.cpu().data, dim=1), test_label.cpu())
    for y1,y2 in zip(y,test_label):
        count_predict[y1]+=1
        count_total[y2]+=1
        print("%d,%d" %(y1,y2))
        if y1==y2:
            count_right[y1]+=1

precision = [0,0]
recall = [0,0]
for i in range(len(count_predict)):
    if count_predict[i]!=0:
          precision[i] = float(count_right[i])/count_predict[i]
          recall[i] = float(count_right[i])/count_total[i]
    
precision = sum(precision)/len(precision)
recall = sum(recall)/len(recall)    
print("loss:%.5f , 准确率：%.4f , 召回率：%.4f , f：%.4f   |准确率 %.4f" %(test_losses.data/m,precision,recall,(2*precision*recall)/(precision+recall),test_acc/m))


0,0
1,1
0,1
0,0
0,0
0,1
0,0
0,0
loss:0.65570 , 准确率：0.8571 , 召回率：0.6667 , f：0.7500   |准确率 0.7500


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.LSTM' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn

In [0]:
#BiLSTM

class Model3(nn.Module):
    def __init__(self,labels,vocab_size,embedding_dim,embedding_dim_tag,weight,tag,hidden_dim,pos_size,pos_dim,batch,num_layers,tag_size,tag_dim):  
        super(Model3,self).__init__()
        self.batch_size = batch
        self.embedding_dim_1 = embedding_dim  #100 model_1
        self.embedding_dim_2 = embedding_dim_tag  #130 model_2
        self.hidden_dim = hidden_dim  
        self.vocab_size = vocab_size  
        self.labels = labels
        self.num_layers = num_layers
        self.pos_size = pos_size  
        self.pos_dim = pos_dim  #25
        self.tag_size = tag_size  
        self.tag_dim = tag_dim  #30
        
        self.bidirectional = True
        self.num_directions = 2 if self.bidirectional else 1#2
        
        self.embedding_1 = nn.Embedding.from_pretrained(weight)
        self.embedding_1.weight.requires_grad = False
        self.tag_embeds = nn.Embedding.from_pretrained(tag)#weight_tag为7*30
        self.tag_embeds.weight.requires_grad = False
        self.pos_embeds = nn.Embedding(self.pos_size,self.pos_dim)#位置特征 pos_dim :25维 
        
        self.lstm_1 = nn.LSTM(input_size=self.embedding_dim_1,
                              hidden_size=self.hidden_dim,
                              num_layers=self.num_layers, 
                              bidirectional=self.bidirectional,
                              dropout=0,
                              batch_first = True
                             )#125b
        self.ln = LayerNorm(self.hidden_dim * self.num_directions)
        self.logistic = nn.Linear(self.hidden_dim * self.num_directions,self.labels)
        
        #if self.bidirectional:
        #    self.decoder2 = nn.Linear(hidden_size * 2, labels)
       # else:
         #   self.decoder2 = nn.Linear(hidden_size * , labels)
        self.dropout=nn.Dropout(p=0.5)
        self.dropout_emb=nn.Dropout(p=0.3)
        
        self._init_weights()
          
    def _init_weights(self, scope=1.):
        self.logistic.weight.data.uniform_(-scope, scope)
        self.logistic.bias.data.fill_(0)

    def init_hidden(self):
        num_layer = self.num_layers * self.num_directions
        weight = next(self.parameters()).data
        return (Variable(weight.new(num_layer, self.batch_size, self.hidden_dim).zero_()), Variable(weight.new(num_layer, self.batch_size, self.hidden_dim).zero_()))

    def forward(self,sentence,pos,tag,hidden):#加tag
        encode = self.embedding_1(sentence)
        #encoding_1 = self.dropout_emb(encode)#对拼接后的层dropout
        lstm_out, hidden = self.lstm_1(encode, hidden)
        
        lstm_out = torch.transpose(lstm_out,0,1)
        lstm_out = F.tanh(lstm_out)
        output = self.ln(lstm_out)[-1]
        output = F.tanh(output)
        return F.log_softmax(self.logistic(output))
      
      
       # embeds_1 = self.embedding_1(sentence)
        #encoding_1 = self.dropout_emb(embeds_1)#对拼接后的层dropout
       # states_1, hidden = self.lstm_1(embeds_1.transpose(0, 1),hidden)
       # print(states_1.size())
      
        #states_1 = torch.transpose(states_1,1,2)
        #encoding_1 = F.tanh(states_1)
     
        #states_1 = F.max_pool1d(states_1,states_1.size(2))
        #states_1 = states_1.squeeze()#2
        #outputs = F.tanh(states_1)
       # encoding_1 = F.dropout(encoding_1, p=0.5)#对拼接后的层dropout
        #outputs = self.decoder(encoding_1)
       # outputs = self.decoder2(encoding_1)
       # outputs = F.log_softmax(self.logistic(outputs))
        #print(outputs.size())
       # return outputs
      

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
--2019-04-29 05:10:56--  https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
Resolving launchpad.net (launchpad.net)... 91.189.89.223, 91.189.89.222, 2001:67c:1560:8003::8004, ...
Connecting to launchpad.net (launchpad.net)|91.189.89.223|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2019-04-29 05:10:57 ERROR 404: Not Found.

dpkg: error: cannot access archive 'google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb': No such file or directory
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 53 not upgraded.


KeyboardInterrupt: ignored

In [0]:
!add-apt-repository -y  ppa:alessandro-strada/ppa

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,609 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/alessandro-strada/ppa/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:1

In [0]:
!apt-get update

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/alessandro-strada/ppa/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (126

In [0]:
!apt install w3m

Reading package lists... Done
Building dependency tree       
Reading state information... Done
w3m is already the newest version (0.5.3-36build1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 53 not upgraded.


In [0]:
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive


fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option
'Colab Notebooks'   python3
